In [10]:
import argparse
import os
import pandas as pd
import snowflake.connector
import json
import boto3
import xxhash
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from sklearn.metrics import roc_auc_score
import numpy as np
from datetime import datetime, timedelta
import xgboost as xgb
from sklearn.metrics import accuracy_score, roc_auc_score

from typing import Dict

from tqdm import tqdm
from code.snowflake_utils import connect


sf_account_id = "lnb99345.us-east-1"
sf_secret_id = "snowflake_credentials"
warehouse = "XSMALL"
database = "PRODUCTION"
schema = "SIGNALS"


In [11]:
ctx = connect(secret_id=sf_secret_id,
              account=sf_account_id, 
              warehouse=warehouse,
              database=database,
              schema=schema, 
              protocol="https",
              region="us-east-1",
              profile_name="ml-staging-admin")
ctx

sf_user=MGAIDUK, sf_password=****, sf_account=lnb99345.us-east-1, sf_warehouse=XSMALL, sf_database=PRODUCTION, sf_schema=SIGNALS, sf_protocol=https


In [12]:
def collect_dataset(ctx: snowflake.connector.SnowflakeConnection, input: str) -> pd.DataFrame:
    # train on a 7-day lookback window
    filter_time = (datetime.now() - timedelta(days=2)).strftime('%Y-%m-%d')
    # Collect dataset
    sql = f"""
    select * from {input}
    where min_timestamp >= '{filter_time}'
    """
    df = pd.read_sql(sql, ctx)
    return df

df = collect_dataset(ctx, input="TRAIN_DATASET_V2")
df.shape, df

/tmp/ipykernel_182773/86426041.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, ctx)


((227163, 168),
                                       USERID                     MEDIAID  \
 0       e246db62-a5db-436b-b46c-401a11e46ec3  01HMGZMZW8VVWK1XT7Q5Z6QR82   
 1       cad4881d-750b-44b0-9fd9-dcdc1be0c045  01HH7MK3HYX98T8Y13YRR7EHC3   
 2       12332299-a650-4437-a32e-5b6b00d186d7  01H79NTA46R5026AF135CWV49T   
 3       fdcd1508-bef6-415e-a295-c6e64009e0f2  01HMC08WN5A8698D55BJNJPRGZ   
 4       407bc575-dcec-49a4-a1b9-36adda7db01a  01HHB011N4PWTB5P6XZETGX1VK   
 ...                                      ...                         ...   
 227158  9bac9fa8-80a4-4b06-b980-63e765e04f83  01HMH2YSJSKNP0QY9RT8AW525M   
 227159  b0711ee2-069d-44f0-bf58-44fc42c04e33  01HMG7FVWGD6CFPGVDDTN129S0   
 227160  451114a8-1419-4f74-9f13-6f2796579677  01HEPF6RT42CX8TF1E3HB35ZJS   
 227161  852599f3-0e9e-4761-8d19-2a95b94a0a9b  01H84QYMCSNJ94E031MSH4HJAK   
 227162  99cbced7-a433-43b2-a5ec-237faf124d16  01HES59DZQ326ANFPSBAETN47H   
 
                               MEDIATAKENBYID  \
 0       

In [13]:
list(df.columns)

['USERID',
 'MEDIAID',
 'MEDIATAKENBYID',
 'REQUESTID',
 'NVIEWS',
 'TIMESPENT',
 'MIN_TIMESTAMP',
 'NREACTIONS',
 'TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_1M',
 'TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_5M',
 'TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_15M',
 'TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_1H',
 'TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_6H',
 'TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_1D',
 'TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_7D',
 'TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_30D',
 'TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_1M',
 'TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_5M',
 'TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_15M',
 'TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_1H',
 'TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_6H',
 'TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_1D',
 'TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_7D',
 'TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_30D',
 'TAKENBY_CFMEDIADWELLTIMESIGNAL_1M',
 'TAKENBY_CFMEDIADWELLTIMESIGNAL_5M',
 'TAKENBY_CFMEDIADWELLTIMESIGNAL_15M',
 'TAKENBY_CFMEDIADWELLTIMESIGNAL_1H',
 'TAKENBY_CFMEDIADWELLTIMESIGNAL_6H',
 'TAKENBY_CFMEDIADWELLTIMES

In [14]:
df["HAS_REACTIONS"] = df["NREACTIONS"] > 0
df["HAS_REACTIONS"]

0         False
1         False
2         False
3         False
4         False
          ...  
227158    False
227159    False
227160    False
227161    False
227162    False
Name: HAS_REACTIONS, Length: 227163, dtype: bool

In [15]:
yesterday = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')

val_df = df[df["MIN_TIMESTAMP"] > yesterday]
train_df = df[df["MIN_TIMESTAMP"] <= yesterday]
train_df.shape, val_df.shape

((156787, 169), (70376, 169))

In [16]:
features = ['TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_1M','TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_5M','TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_15M','TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_1H','TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_6H','TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_1D','TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_7D','TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_30D','TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_1M','TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_5M','TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_15M','TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_1H','TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_6H','TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_1D','TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_7D','TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_30D','TAKENBY_CFMEDIADWELLTIMESIGNAL_1M','TAKENBY_CFMEDIADWELLTIMESIGNAL_5M','TAKENBY_CFMEDIADWELLTIMESIGNAL_15M','TAKENBY_CFMEDIADWELLTIMESIGNAL_1H','TAKENBY_CFMEDIADWELLTIMESIGNAL_6H','TAKENBY_CFMEDIADWELLTIMESIGNAL_1D','TAKENBY_CFMEDIADWELLTIMESIGNAL_7D','TAKENBY_CFMEDIADWELLTIMESIGNAL_30D','TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_1M','TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_5M','TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_15M','TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_1H','TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_6H','TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_1D','TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_7D','TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_30D','TAKENBY_CFMEDIAVIEWEDSIGNAL_1M','TAKENBY_CFMEDIAVIEWEDSIGNAL_5M','TAKENBY_CFMEDIAVIEWEDSIGNAL_15M','TAKENBY_CFMEDIAVIEWEDSIGNAL_1H','TAKENBY_CFMEDIAVIEWEDSIGNAL_6H','TAKENBY_CFMEDIAVIEWEDSIGNAL_1D','TAKENBY_CFMEDIAVIEWEDSIGNAL_7D','TAKENBY_CFMEDIAVIEWEDSIGNAL_30D','MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_1M','MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_5M','MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_15M','MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_1H','MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_6H','MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_1D','MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_7D','MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_30D','MEDIA_CFMEDIAADDEDREACTIONSIGNAL_1M','MEDIA_CFMEDIAADDEDREACTIONSIGNAL_5M','MEDIA_CFMEDIAADDEDREACTIONSIGNAL_15M','MEDIA_CFMEDIAADDEDREACTIONSIGNAL_1H','MEDIA_CFMEDIAADDEDREACTIONSIGNAL_6H','MEDIA_CFMEDIAADDEDREACTIONSIGNAL_1D','MEDIA_CFMEDIAADDEDREACTIONSIGNAL_7D','MEDIA_CFMEDIAADDEDREACTIONSIGNAL_30D','MEDIA_CFMEDIADWELLTIMESIGNAL_1M','MEDIA_CFMEDIADWELLTIMESIGNAL_5M','MEDIA_CFMEDIADWELLTIMESIGNAL_15M','MEDIA_CFMEDIADWELLTIMESIGNAL_1H','MEDIA_CFMEDIADWELLTIMESIGNAL_6H','MEDIA_CFMEDIADWELLTIMESIGNAL_1D','MEDIA_CFMEDIADWELLTIMESIGNAL_7D','MEDIA_CFMEDIADWELLTIMESIGNAL_30D','MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_1M','MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_5M','MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_15M','MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_1H','MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_6H','MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_1D','MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_7D','MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_30D','MEDIA_CFMEDIAVIEWEDSIGNAL_1M','MEDIA_CFMEDIAVIEWEDSIGNAL_5M','MEDIA_CFMEDIAVIEWEDSIGNAL_15M','MEDIA_CFMEDIAVIEWEDSIGNAL_1H','MEDIA_CFMEDIAVIEWEDSIGNAL_6H','MEDIA_CFMEDIAVIEWEDSIGNAL_1D','MEDIA_CFMEDIAVIEWEDSIGNAL_7D','MEDIA_CFMEDIAVIEWEDSIGNAL_30D','USER_CFMEDIAADDEDCOMMENTSIGNAL_1M','USER_CFMEDIAADDEDCOMMENTSIGNAL_5M','USER_CFMEDIAADDEDCOMMENTSIGNAL_15M','USER_CFMEDIAADDEDCOMMENTSIGNAL_1H','USER_CFMEDIAADDEDCOMMENTSIGNAL_6H','USER_CFMEDIAADDEDCOMMENTSIGNAL_1D','USER_CFMEDIAADDEDCOMMENTSIGNAL_7D','USER_CFMEDIAADDEDCOMMENTSIGNAL_30D','USER_CFMEDIAADDEDREACTIONSIGNAL_1M','USER_CFMEDIAADDEDREACTIONSIGNAL_5M','USER_CFMEDIAADDEDREACTIONSIGNAL_15M','USER_CFMEDIAADDEDREACTIONSIGNAL_1H','USER_CFMEDIAADDEDREACTIONSIGNAL_6H','USER_CFMEDIAADDEDREACTIONSIGNAL_1D','USER_CFMEDIAADDEDREACTIONSIGNAL_7D','USER_CFMEDIAADDEDREACTIONSIGNAL_30D','USER_CFMEDIADWELLTIMESIGNAL_1M','USER_CFMEDIADWELLTIMESIGNAL_5M','USER_CFMEDIADWELLTIMESIGNAL_15M','USER_CFMEDIADWELLTIMESIGNAL_1H','USER_CFMEDIADWELLTIMESIGNAL_6H','USER_CFMEDIADWELLTIMESIGNAL_1D','USER_CFMEDIADWELLTIMESIGNAL_7D','USER_CFMEDIADWELLTIMESIGNAL_30D','USER_CFMEDIATAPPEDTAKENBYSIGNAL_1M','USER_CFMEDIATAPPEDTAKENBYSIGNAL_5M','USER_CFMEDIATAPPEDTAKENBYSIGNAL_15M','USER_CFMEDIATAPPEDTAKENBYSIGNAL_1H','USER_CFMEDIATAPPEDTAKENBYSIGNAL_6H','USER_CFMEDIATAPPEDTAKENBYSIGNAL_1D','USER_CFMEDIATAPPEDTAKENBYSIGNAL_7D','USER_CFMEDIATAPPEDTAKENBYSIGNAL_30D','USER_CFMEDIAVIEWEDSIGNAL_1M','USER_CFMEDIAVIEWEDSIGNAL_5M','USER_CFMEDIAVIEWEDSIGNAL_15M','USER_CFMEDIAVIEWEDSIGNAL_1H','USER_CFMEDIAVIEWEDSIGNAL_6H','USER_CFMEDIAVIEWEDSIGNAL_1D','USER_CFMEDIAVIEWEDSIGNAL_7D','USER_CFMEDIAVIEWEDSIGNAL_30D','CROSS_CFMEDIAADDEDCOMMENTSIGNAL_1M','CROSS_CFMEDIAADDEDCOMMENTSIGNAL_5M','CROSS_CFMEDIAADDEDCOMMENTSIGNAL_15M','CROSS_CFMEDIAADDEDCOMMENTSIGNAL_1H','CROSS_CFMEDIAADDEDCOMMENTSIGNAL_6H','CROSS_CFMEDIAADDEDCOMMENTSIGNAL_1D','CROSS_CFMEDIAADDEDCOMMENTSIGNAL_7D','CROSS_CFMEDIAADDEDCOMMENTSIGNAL_30D','CROSS_CFMEDIAADDEDREACTIONSIGNAL_1M','CROSS_CFMEDIAADDEDREACTIONSIGNAL_5M','CROSS_CFMEDIAADDEDREACTIONSIGNAL_15M','CROSS_CFMEDIAADDEDREACTIONSIGNAL_1H','CROSS_CFMEDIAADDEDREACTIONSIGNAL_6H','CROSS_CFMEDIAADDEDREACTIONSIGNAL_1D','CROSS_CFMEDIAADDEDREACTIONSIGNAL_7D','CROSS_CFMEDIAADDEDREACTIONSIGNAL_30D','CROSS_CFMEDIADWELLTIMESIGNAL_1M','CROSS_CFMEDIADWELLTIMESIGNAL_5M','CROSS_CFMEDIADWELLTIMESIGNAL_15M','CROSS_CFMEDIADWELLTIMESIGNAL_1H','CROSS_CFMEDIADWELLTIMESIGNAL_6H','CROSS_CFMEDIADWELLTIMESIGNAL_1D','CROSS_CFMEDIADWELLTIMESIGNAL_7D','CROSS_CFMEDIADWELLTIMESIGNAL_30D','CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_1M','CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_5M','CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_15M','CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_1H','CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_6H','CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_1D','CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_7D','CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_30D','CROSS_CFMEDIAVIEWEDSIGNAL_1M','CROSS_CFMEDIAVIEWEDSIGNAL_5M','CROSS_CFMEDIAVIEWEDSIGNAL_15M','CROSS_CFMEDIAVIEWEDSIGNAL_1H','CROSS_CFMEDIAVIEWEDSIGNAL_6H','CROSS_CFMEDIAVIEWEDSIGNAL_1D','CROSS_CFMEDIAVIEWEDSIGNAL_7D','CROSS_CFMEDIAVIEWEDSIGNAL_30D']
target = "HAS_REACTIONS"

In [17]:
features

['TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_1M',
 'TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_5M',
 'TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_15M',
 'TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_1H',
 'TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_6H',
 'TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_1D',
 'TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_7D',
 'TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_30D',
 'TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_1M',
 'TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_5M',
 'TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_15M',
 'TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_1H',
 'TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_6H',
 'TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_1D',
 'TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_7D',
 'TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_30D',
 'TAKENBY_CFMEDIADWELLTIMESIGNAL_1M',
 'TAKENBY_CFMEDIADWELLTIMESIGNAL_5M',
 'TAKENBY_CFMEDIADWELLTIMESIGNAL_15M',
 'TAKENBY_CFMEDIADWELLTIMESIGNAL_1H',
 'TAKENBY_CFMEDIADWELLTIMESIGNAL_6H',
 'TAKENBY_CFMEDIADWELLTIMESIGNAL_1D',
 'TAKENBY_CFMEDIADWELLTIMESIGNAL_7D',
 'TAKENBY_CFMEDIADWELLTIMESIGNAL_30D',
 'TAKENBY_CFMEDIATAPPEDTAK

In [18]:
dtrain = xgb.DMatrix(train_df[features], label=train_df[target])
dval = xgb.DMatrix(val_df[features], label=val_df[target])

In [19]:
params = {
    'objective': 'binary:logistic',  # Binary classification
    'max_depth': 6,                  # Depth of each tree
    'alpha': 1,                     # L1 regularization term on weights
    'learning_rate': 0.05,           # Step size shrinkage used in update to prevents overfitting
}
evals = [(dtrain, 'train'), (dval, 'val')]
eval_metric = ['error', 'auc']
bst = xgb.train(params, dtrain, num_boost_round=100, evals=evals)
y_pred = bst.predict(dval)
auc_score = roc_auc_score(val_df[target], y_pred)
print(f"Validation AUC: {auc_score:.3f}")

[0]	train-logloss:0.20966	val-logloss:0.21236
[1]	train-logloss:0.20461	val-logloss:0.20760
[2]	train-logloss:0.19994	val-logloss:0.20317
[3]	train-logloss:0.19554	val-logloss:0.19915
[4]	train-logloss:0.19145	val-logloss:0.19538
[5]	train-logloss:0.18765	val-logloss:0.19178
[6]	train-logloss:0.18406	val-logloss:0.18850
[7]	train-logloss:0.18072	val-logloss:0.18546
[8]	train-logloss:0.17760	val-logloss:0.18248
[9]	train-logloss:0.17466	val-logloss:0.17981
[10]	train-logloss:0.17190	val-logloss:0.17730
[11]	train-logloss:0.16931	val-logloss:0.17493
[12]	train-logloss:0.16688	val-logloss:0.17269
[13]	train-logloss:0.16460	val-logloss:0.17060
[14]	train-logloss:0.16247	val-logloss:0.16864
[15]	train-logloss:0.16047	val-logloss:0.16684
[16]	train-logloss:0.15860	val-logloss:0.16512
[17]	train-logloss:0.15688	val-logloss:0.16352
[18]	train-logloss:0.15524	val-logloss:0.16203
[19]	train-logloss:0.15371	val-logloss:0.16063
[20]	train-logloss:0.15229	val-logloss:0.15935
[21]	train-logloss:0.15

In [28]:
import json
js = json.dumps(val_df[:2][features].to_dict(orient="records"))
js

'[{"TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_1M": NaN, "TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_5M": NaN, "TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_15M": NaN, "TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_1H": NaN, "TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_6H": NaN, "TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_1D": NaN, "TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_7D": NaN, "TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_30D": NaN, "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_1M": 7.696535994627496e-70, "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_5M": 2.618683775587088e-14, "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_15M": 4.7139070665777155e-05, "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_1H": 0.13935345825584178, "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_6H": 1.3060443652450713, "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_1D": 4.087081774077868, "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_7D": 24.170414981516267, "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_30D": 47.9210113498678, "TAKENBY_CFMEDIADWELLTIMESIGNAL_1M": 8.538857909291201e-05, "TAKENBY_CFMEDIADWELLTIMESIGNAL_5M": 0.3284413956793213, "TAKENBY_CFMEDIADWEL

In [21]:
js = """
[
    {
        "TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_1M": 0.0,
        "TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_5M": 0.0,
        "TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_15M": 3.1995214417109996E-217,
        "TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_1H": 1.2648637697216277E-54,
        "TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_6H": 1.852958198365319E-9,
        "TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_1D": 0.011060888258594052,
        "TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_7D": 1.173711878402293,
        "TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_30D": 2.3857168175300303,
        "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_1M": 6.078168951713034E-163,
        "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_5M": 3.603755393911616E-33,
        "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_15M": 1.5515600563641378E-11,
        "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_1H": 0.002633887814234353,
        "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_6H": 0.773526871494684,
        "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_1D": 4.714830933849994,
        "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_7D": 36.200576557672726,
        "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_30D": 72.16713015616625,
        "TAKENBY_CFMEDIADWELLTIMESIGNAL_1M": 0.0000024892673197384333,
        "TAKENBY_CFMEDIADWELLTIMESIGNAL_5M": 0.18450892828456525,
        "TAKENBY_CFMEDIADWELLTIMESIGNAL_15M": 1.268531651961793,
        "TAKENBY_CFMEDIADWELLTIMESIGNAL_1H": 8.588301047377449,
        "TAKENBY_CFMEDIADWELLTIMESIGNAL_6H": 71.2359075332376,
        "TAKENBY_CFMEDIADWELLTIMESIGNAL_1D": 234.71703750681183,
        "TAKENBY_CFMEDIADWELLTIMESIGNAL_7D": 3470.0852808737377,
        "TAKENBY_CFMEDIADWELLTIMESIGNAL_30D": 7805.433228058954,
        "TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_1M": 5.7715289329696065E-201,
        "TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_5M": 8.958975323772849E-41,
        "TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_15M": 4.526025182431397E-14,
        "TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_1H": 0.0007132050871835041,
        "TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_6H": 0.7523556442993248,
        "TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_1D": 4.101445607385634,
        "TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_7D": 20.953016430402073,
        "TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_30D": 40.742379877529714,
        "TAKENBY_CFMEDIAVIEWEDSIGNAL_1M": 8.0426796514209E-7,
        "TAKENBY_CFMEDIAVIEWEDSIGNAL_5M": 0.060848986737352534,
        "TAKENBY_CFMEDIAVIEWEDSIGNAL_15M": 0.5317893562868538,
        "TAKENBY_CFMEDIAVIEWEDSIGNAL_1H": 4.487116821653351,
        "TAKENBY_CFMEDIAVIEWEDSIGNAL_6H": 27.37048022256756,
        "TAKENBY_CFMEDIAVIEWEDSIGNAL_1D": 78.8725141138884,
        "TAKENBY_CFMEDIAVIEWEDSIGNAL_7D": 504.231241929909,
        "TAKENBY_CFMEDIAVIEWEDSIGNAL_30D": 1058.261218450494,
        "MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_1M": 0.0,
        "MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_5M": 0.0,
        "MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_15M": 0.0,
        "MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_1H": 0.0,
        "MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_6H": 0.0,
        "MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_1D": 0.0,
        "MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_7D": 0.0,
        "MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_30D": 0.0,
        "MEDIA_CFMEDIAADDEDREACTIONSIGNAL_1M": 6.078168951713034E-163,
        "MEDIA_CFMEDIAADDEDREACTIONSIGNAL_5M": 6.078168951713034E-163,
        "MEDIA_CFMEDIAADDEDREACTIONSIGNAL_15M": 6.078168951713034E-163,
        "MEDIA_CFMEDIAADDEDREACTIONSIGNAL_1H": 6.078168951713034E-163,
        "MEDIA_CFMEDIAADDEDREACTIONSIGNAL_6H": 6.078168951713034E-163,
        "MEDIA_CFMEDIAADDEDREACTIONSIGNAL_1D": 6.078168951713034E-163,
        "MEDIA_CFMEDIAADDEDREACTIONSIGNAL_7D": 6.078168951713034E-163,
        "MEDIA_CFMEDIAADDEDREACTIONSIGNAL_30D": 6.078168951713034E-163,
        "MEDIA_CFMEDIADWELLTIMESIGNAL_1M": 0.0000024892673197384333,
        "MEDIA_CFMEDIADWELLTIMESIGNAL_5M": 0.0000024892673197384333,
        "MEDIA_CFMEDIADWELLTIMESIGNAL_15M": 0.0000024892673197384333,
        "MEDIA_CFMEDIADWELLTIMESIGNAL_1H": 0.0000024892673197384333,
        "MEDIA_CFMEDIADWELLTIMESIGNAL_6H": 0.0000024892673197384333,
        "MEDIA_CFMEDIADWELLTIMESIGNAL_1D": 0.0000024892673197384333,
        "MEDIA_CFMEDIADWELLTIMESIGNAL_7D": 0.0000024892673197384333,
        "MEDIA_CFMEDIADWELLTIMESIGNAL_30D": 0.0000024892673197384333,
        "MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_1M": 3.641708684270539E-68,
        "MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_5M": 3.641708684270539E-68,
        "MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_15M": 3.641708684270539E-68,
        "MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_1H": 3.641708684270539E-68,
        "MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_6H": 3.641708684270539E-68,
        "MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_1D": 3.641708684270539E-68,
        "MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_7D": 3.641708684270539E-68,
        "MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_30D": 3.641708684270539E-68,
        "MEDIA_CFMEDIAVIEWEDSIGNAL_1M": 8.0426796514209E-7,
        "MEDIA_CFMEDIAVIEWEDSIGNAL_5M": 8.0426796514209E-7,
        "MEDIA_CFMEDIAVIEWEDSIGNAL_15M": 8.0426796514209E-7,
        "MEDIA_CFMEDIAVIEWEDSIGNAL_1H": 8.0426796514209E-7,
        "MEDIA_CFMEDIAVIEWEDSIGNAL_6H": 8.0426796514209E-7,
        "MEDIA_CFMEDIAVIEWEDSIGNAL_1D": 8.0426796514209E-7,
        "MEDIA_CFMEDIAVIEWEDSIGNAL_7D": 8.0426796514209E-7,
        "MEDIA_CFMEDIAVIEWEDSIGNAL_30D": 8.0426796514209E-7,
        "USER_CFMEDIAADDEDCOMMENTSIGNAL_1M": NaN,
        "USER_CFMEDIAADDEDCOMMENTSIGNAL_5M": NaN,
        "USER_CFMEDIAADDEDCOMMENTSIGNAL_15M": NaN,
        "USER_CFMEDIAADDEDCOMMENTSIGNAL_1H": NaN,
        "USER_CFMEDIAADDEDCOMMENTSIGNAL_6H": NaN,
        "USER_CFMEDIAADDEDCOMMENTSIGNAL_1D": NaN,
        "USER_CFMEDIAADDEDCOMMENTSIGNAL_7D": NaN,
        "USER_CFMEDIAADDEDCOMMENTSIGNAL_30D": NaN,
        "USER_CFMEDIAADDEDREACTIONSIGNAL_1M": NaN,
        "USER_CFMEDIAADDEDREACTIONSIGNAL_5M": NaN,
        "USER_CFMEDIAADDEDREACTIONSIGNAL_15M": NaN,
        "USER_CFMEDIAADDEDREACTIONSIGNAL_1H": NaN,
        "USER_CFMEDIAADDEDREACTIONSIGNAL_6H": NaN,
        "USER_CFMEDIAADDEDREACTIONSIGNAL_1D": NaN,
        "USER_CFMEDIAADDEDREACTIONSIGNAL_7D": NaN,
        "USER_CFMEDIAADDEDREACTIONSIGNAL_30D": NaN,
        "USER_CFMEDIADWELLTIMESIGNAL_1M": NaN,
        "USER_CFMEDIADWELLTIMESIGNAL_5M": NaN,
        "USER_CFMEDIADWELLTIMESIGNAL_15M": NaN,
        "USER_CFMEDIADWELLTIMESIGNAL_1H": NaN,
        "USER_CFMEDIADWELLTIMESIGNAL_6H": NaN,
        "USER_CFMEDIADWELLTIMESIGNAL_1D": NaN,
        "USER_CFMEDIADWELLTIMESIGNAL_7D": NaN,
        "USER_CFMEDIADWELLTIMESIGNAL_30D": NaN,
        "USER_CFMEDIATAPPEDTAKENBYSIGNAL_1M": NaN,
        "USER_CFMEDIATAPPEDTAKENBYSIGNAL_5M": NaN,
        "USER_CFMEDIATAPPEDTAKENBYSIGNAL_15M": NaN,
        "USER_CFMEDIATAPPEDTAKENBYSIGNAL_1H": NaN,
        "USER_CFMEDIATAPPEDTAKENBYSIGNAL_6H": NaN,
        "USER_CFMEDIATAPPEDTAKENBYSIGNAL_1D": NaN,
        "USER_CFMEDIATAPPEDTAKENBYSIGNAL_7D": NaN,
        "USER_CFMEDIATAPPEDTAKENBYSIGNAL_30D": NaN,
        "USER_CFMEDIAVIEWEDSIGNAL_1M": NaN,
        "USER_CFMEDIAVIEWEDSIGNAL_5M": NaN,
        "USER_CFMEDIAVIEWEDSIGNAL_15M": NaN,
        "USER_CFMEDIAVIEWEDSIGNAL_1H": NaN,
        "USER_CFMEDIAVIEWEDSIGNAL_6H": NaN,
        "USER_CFMEDIAVIEWEDSIGNAL_1D": NaN,
        "USER_CFMEDIAVIEWEDSIGNAL_7D": NaN,
        "USER_CFMEDIAVIEWEDSIGNAL_30D": NaN,
        "CROSS_CFMEDIAADDEDCOMMENTSIGNAL_1M": NaN,
        "CROSS_CFMEDIAADDEDCOMMENTSIGNAL_5M": NaN,
        "CROSS_CFMEDIAADDEDCOMMENTSIGNAL_15M": NaN,
        "CROSS_CFMEDIAADDEDCOMMENTSIGNAL_1H": NaN,
        "CROSS_CFMEDIAADDEDCOMMENTSIGNAL_6H": NaN,
        "CROSS_CFMEDIAADDEDCOMMENTSIGNAL_1D": NaN,
        "CROSS_CFMEDIAADDEDCOMMENTSIGNAL_7D": NaN,
        "CROSS_CFMEDIAADDEDCOMMENTSIGNAL_30D": NaN,
        "CROSS_CFMEDIAADDEDREACTIONSIGNAL_1M": NaN,
        "CROSS_CFMEDIAADDEDREACTIONSIGNAL_5M": NaN,
        "CROSS_CFMEDIAADDEDREACTIONSIGNAL_15M": NaN,
        "CROSS_CFMEDIAADDEDREACTIONSIGNAL_1H": NaN,
        "CROSS_CFMEDIAADDEDREACTIONSIGNAL_6H": NaN,
        "CROSS_CFMEDIAADDEDREACTIONSIGNAL_1D": NaN,
        "CROSS_CFMEDIAADDEDREACTIONSIGNAL_7D": NaN,
        "CROSS_CFMEDIAADDEDREACTIONSIGNAL_30D": NaN,
        "CROSS_CFMEDIADWELLTIMESIGNAL_1M": NaN,
        "CROSS_CFMEDIADWELLTIMESIGNAL_5M": NaN,
        "CROSS_CFMEDIADWELLTIMESIGNAL_15M": NaN,
        "CROSS_CFMEDIADWELLTIMESIGNAL_1H": NaN,
        "CROSS_CFMEDIADWELLTIMESIGNAL_6H": NaN,
        "CROSS_CFMEDIADWELLTIMESIGNAL_1D": NaN,
        "CROSS_CFMEDIADWELLTIMESIGNAL_7D": NaN,
        "CROSS_CFMEDIADWELLTIMESIGNAL_30D": NaN,
        "CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_1M": NaN,
        "CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_5M": NaN,
        "CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_15M": NaN,
        "CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_1H": NaN,
        "CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_6H": NaN,
        "CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_1D": NaN,
        "CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_7D": NaN,
        "CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_30D": NaN,
        "CROSS_CFMEDIAVIEWEDSIGNAL_1M": NaN,
        "CROSS_CFMEDIAVIEWEDSIGNAL_5M": NaN,
        "CROSS_CFMEDIAVIEWEDSIGNAL_15M": NaN,
        "CROSS_CFMEDIAVIEWEDSIGNAL_1H": NaN,
        "CROSS_CFMEDIAVIEWEDSIGNAL_6H": NaN,
        "CROSS_CFMEDIAVIEWEDSIGNAL_1D": NaN,
        "CROSS_CFMEDIAVIEWEDSIGNAL_7D": NaN,
        "CROSS_CFMEDIAVIEWEDSIGNAL_30D": NaN
    },
    {
        "TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_1M": NaN,
        "TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_5M": NaN,
        "TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_15M": NaN,
        "TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_1H": NaN,
        "TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_6H": NaN,
        "TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_1D": NaN,
        "TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_7D": NaN,
        "TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_30D": NaN,
        "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_1M": NaN,
        "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_5M": NaN,
        "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_15M": NaN,
        "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_1H": NaN,
        "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_6H": NaN,
        "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_1D": NaN,
        "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_7D": NaN,
        "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_30D": NaN,
        "TAKENBY_CFMEDIADWELLTIMESIGNAL_1M": NaN,
        "TAKENBY_CFMEDIADWELLTIMESIGNAL_5M": NaN,
        "TAKENBY_CFMEDIADWELLTIMESIGNAL_15M": NaN,
        "TAKENBY_CFMEDIADWELLTIMESIGNAL_1H": NaN,
        "TAKENBY_CFMEDIADWELLTIMESIGNAL_6H": NaN,
        "TAKENBY_CFMEDIADWELLTIMESIGNAL_1D": NaN,
        "TAKENBY_CFMEDIADWELLTIMESIGNAL_7D": NaN,
        "TAKENBY_CFMEDIADWELLTIMESIGNAL_30D": NaN,
        "TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_1M": NaN,
        "TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_5M": NaN,
        "TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_15M": NaN,
        "TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_1H": NaN,
        "TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_6H": NaN,
        "TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_1D": NaN,
        "TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_7D": NaN,
        "TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_30D": NaN,
        "TAKENBY_CFMEDIAVIEWEDSIGNAL_1M": NaN,
        "TAKENBY_CFMEDIAVIEWEDSIGNAL_5M": NaN,
        "TAKENBY_CFMEDIAVIEWEDSIGNAL_15M": NaN,
        "TAKENBY_CFMEDIAVIEWEDSIGNAL_1H": NaN,
        "TAKENBY_CFMEDIAVIEWEDSIGNAL_6H": NaN,
        "TAKENBY_CFMEDIAVIEWEDSIGNAL_1D": NaN,
        "TAKENBY_CFMEDIAVIEWEDSIGNAL_7D": NaN,
        "TAKENBY_CFMEDIAVIEWEDSIGNAL_30D": NaN,
        "MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_1M": NaN,
        "MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_5M": NaN,
        "MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_15M": NaN,
        "MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_1H": NaN,
        "MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_6H": NaN,
        "MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_1D": NaN,
        "MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_7D": NaN,
        "MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_30D": NaN,
        "MEDIA_CFMEDIAADDEDREACTIONSIGNAL_1M": NaN,
        "MEDIA_CFMEDIAADDEDREACTIONSIGNAL_5M": NaN,
        "MEDIA_CFMEDIAADDEDREACTIONSIGNAL_15M": NaN,
        "MEDIA_CFMEDIAADDEDREACTIONSIGNAL_1H": NaN,
        "MEDIA_CFMEDIAADDEDREACTIONSIGNAL_6H": NaN,
        "MEDIA_CFMEDIAADDEDREACTIONSIGNAL_1D": NaN,
        "MEDIA_CFMEDIAADDEDREACTIONSIGNAL_7D": NaN,
        "MEDIA_CFMEDIAADDEDREACTIONSIGNAL_30D": NaN,
        "MEDIA_CFMEDIADWELLTIMESIGNAL_1M": NaN,
        "MEDIA_CFMEDIADWELLTIMESIGNAL_5M": NaN,
        "MEDIA_CFMEDIADWELLTIMESIGNAL_15M": NaN,
        "MEDIA_CFMEDIADWELLTIMESIGNAL_1H": NaN,
        "MEDIA_CFMEDIADWELLTIMESIGNAL_6H": NaN,
        "MEDIA_CFMEDIADWELLTIMESIGNAL_1D": NaN,
        "MEDIA_CFMEDIADWELLTIMESIGNAL_7D": NaN,
        "MEDIA_CFMEDIADWELLTIMESIGNAL_30D": NaN,
        "MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_1M": NaN,
        "MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_5M": NaN,
        "MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_15M": NaN,
        "MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_1H": NaN,
        "MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_6H": NaN,
        "MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_1D": NaN,
        "MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_7D": NaN,
        "MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_30D": NaN,
        "MEDIA_CFMEDIAVIEWEDSIGNAL_1M": NaN,
        "MEDIA_CFMEDIAVIEWEDSIGNAL_5M": NaN,
        "MEDIA_CFMEDIAVIEWEDSIGNAL_15M": NaN,
        "MEDIA_CFMEDIAVIEWEDSIGNAL_1H": NaN,
        "MEDIA_CFMEDIAVIEWEDSIGNAL_6H": NaN,
        "MEDIA_CFMEDIAVIEWEDSIGNAL_1D": NaN,
        "MEDIA_CFMEDIAVIEWEDSIGNAL_7D": NaN,
        "MEDIA_CFMEDIAVIEWEDSIGNAL_30D": NaN,
        "USER_CFMEDIAADDEDCOMMENTSIGNAL_1M": NaN,
        "USER_CFMEDIAADDEDCOMMENTSIGNAL_5M": NaN,
        "USER_CFMEDIAADDEDCOMMENTSIGNAL_15M": NaN,
        "USER_CFMEDIAADDEDCOMMENTSIGNAL_1H": NaN,
        "USER_CFMEDIAADDEDCOMMENTSIGNAL_6H": NaN,
        "USER_CFMEDIAADDEDCOMMENTSIGNAL_1D": NaN,
        "USER_CFMEDIAADDEDCOMMENTSIGNAL_7D": NaN,
        "USER_CFMEDIAADDEDCOMMENTSIGNAL_30D": NaN,
        "USER_CFMEDIAADDEDREACTIONSIGNAL_1M": NaN,
        "USER_CFMEDIAADDEDREACTIONSIGNAL_5M": NaN,
        "USER_CFMEDIAADDEDREACTIONSIGNAL_15M": NaN,
        "USER_CFMEDIAADDEDREACTIONSIGNAL_1H": NaN,
        "USER_CFMEDIAADDEDREACTIONSIGNAL_6H": NaN,
        "USER_CFMEDIAADDEDREACTIONSIGNAL_1D": NaN,
        "USER_CFMEDIAADDEDREACTIONSIGNAL_7D": NaN,
        "USER_CFMEDIAADDEDREACTIONSIGNAL_30D": NaN,
        "USER_CFMEDIADWELLTIMESIGNAL_1M": NaN,
        "USER_CFMEDIADWELLTIMESIGNAL_5M": NaN,
        "USER_CFMEDIADWELLTIMESIGNAL_15M": NaN,
        "USER_CFMEDIADWELLTIMESIGNAL_1H": NaN,
        "USER_CFMEDIADWELLTIMESIGNAL_6H": NaN,
        "USER_CFMEDIADWELLTIMESIGNAL_1D": NaN,
        "USER_CFMEDIADWELLTIMESIGNAL_7D": NaN,
        "USER_CFMEDIADWELLTIMESIGNAL_30D": NaN,
        "USER_CFMEDIATAPPEDTAKENBYSIGNAL_1M": NaN,
        "USER_CFMEDIATAPPEDTAKENBYSIGNAL_5M": NaN,
        "USER_CFMEDIATAPPEDTAKENBYSIGNAL_15M": NaN,
        "USER_CFMEDIATAPPEDTAKENBYSIGNAL_1H": NaN,
        "USER_CFMEDIATAPPEDTAKENBYSIGNAL_6H": NaN,
        "USER_CFMEDIATAPPEDTAKENBYSIGNAL_1D": NaN,
        "USER_CFMEDIATAPPEDTAKENBYSIGNAL_7D": NaN,
        "USER_CFMEDIATAPPEDTAKENBYSIGNAL_30D": NaN,
        "USER_CFMEDIAVIEWEDSIGNAL_1M": NaN,
        "USER_CFMEDIAVIEWEDSIGNAL_5M": NaN,
        "USER_CFMEDIAVIEWEDSIGNAL_15M": NaN,
        "USER_CFMEDIAVIEWEDSIGNAL_1H": NaN,
        "USER_CFMEDIAVIEWEDSIGNAL_6H": NaN,
        "USER_CFMEDIAVIEWEDSIGNAL_1D": NaN,
        "USER_CFMEDIAVIEWEDSIGNAL_7D": NaN,
        "USER_CFMEDIAVIEWEDSIGNAL_30D": NaN,
        "CROSS_CFMEDIAADDEDCOMMENTSIGNAL_1M": NaN,
        "CROSS_CFMEDIAADDEDCOMMENTSIGNAL_5M": NaN,
        "CROSS_CFMEDIAADDEDCOMMENTSIGNAL_15M": NaN,
        "CROSS_CFMEDIAADDEDCOMMENTSIGNAL_1H": NaN,
        "CROSS_CFMEDIAADDEDCOMMENTSIGNAL_6H": NaN,
        "CROSS_CFMEDIAADDEDCOMMENTSIGNAL_1D": NaN,
        "CROSS_CFMEDIAADDEDCOMMENTSIGNAL_7D": NaN,
        "CROSS_CFMEDIAADDEDCOMMENTSIGNAL_30D": NaN,
        "CROSS_CFMEDIAADDEDREACTIONSIGNAL_1M": NaN,
        "CROSS_CFMEDIAADDEDREACTIONSIGNAL_5M": NaN,
        "CROSS_CFMEDIAADDEDREACTIONSIGNAL_15M": NaN,
        "CROSS_CFMEDIAADDEDREACTIONSIGNAL_1H": NaN,
        "CROSS_CFMEDIAADDEDREACTIONSIGNAL_6H": NaN,
        "CROSS_CFMEDIAADDEDREACTIONSIGNAL_1D": NaN,
        "CROSS_CFMEDIAADDEDREACTIONSIGNAL_7D": NaN,
        "CROSS_CFMEDIAADDEDREACTIONSIGNAL_30D": NaN,
        "CROSS_CFMEDIADWELLTIMESIGNAL_1M": NaN,
        "CROSS_CFMEDIADWELLTIMESIGNAL_5M": NaN,
        "CROSS_CFMEDIADWELLTIMESIGNAL_15M": NaN,
        "CROSS_CFMEDIADWELLTIMESIGNAL_1H": NaN,
        "CROSS_CFMEDIADWELLTIMESIGNAL_6H": NaN,
        "CROSS_CFMEDIADWELLTIMESIGNAL_1D": NaN,
        "CROSS_CFMEDIADWELLTIMESIGNAL_7D": NaN,
        "CROSS_CFMEDIADWELLTIMESIGNAL_30D": NaN,
        "CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_1M": NaN,
        "CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_5M": NaN,
        "CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_15M": NaN,
        "CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_1H": NaN,
        "CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_6H": NaN,
        "CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_1D": NaN,
        "CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_7D": NaN,
        "CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_30D": NaN,
        "CROSS_CFMEDIAVIEWEDSIGNAL_1M": NaN,
        "CROSS_CFMEDIAVIEWEDSIGNAL_5M": NaN,
        "CROSS_CFMEDIAVIEWEDSIGNAL_15M": NaN,
        "CROSS_CFMEDIAVIEWEDSIGNAL_1H": NaN,
        "CROSS_CFMEDIAVIEWEDSIGNAL_6H": NaN,
        "CROSS_CFMEDIAVIEWEDSIGNAL_1D": NaN,
        "CROSS_CFMEDIAVIEWEDSIGNAL_7D": NaN,
        "CROSS_CFMEDIAVIEWEDSIGNAL_30D": NaN
    }
]
"""
dt = pd.DataFrame(json.loads(js))
dt

,TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_1M,TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_5M,TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_15M,TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_1H,TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_6H,TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_1D,TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_7D,TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_30D,TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_1M,TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_5M,...,CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_7D,CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_30D,CROSS_CFMEDIAVIEWEDSIGNAL_1M,CROSS_CFMEDIAVIEWEDSIGNAL_5M,CROSS_CFMEDIAVIEWEDSIGNAL_15M,CROSS_CFMEDIAVIEWEDSIGNAL_1H,CROSS_CFMEDIAVIEWEDSIGNAL_6H,CROSS_CFMEDIAVIEWEDSIGNAL_1D,CROSS_CFMEDIAVIEWEDSIGNAL_7D,CROSS_CFMEDIAVIEWEDSIGNAL_30D
0,0.0,0.0,3.199521e-217,1.264864e-54,1.852958e-09,0.011061,1.173712,2.385717,6.078169e-163,3.603755e-33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
dmatrix = xgb.DMatrix(dt)
predictions = bst.predict(dmatrix)
predictions

array([0.02219246, 0.03387076], dtype=float32)

In [19]:
json.dumps(predictions.tolist())

'[0.016385870054364204, 0.031418099999427795]'

In [27]:
dtrain.feature_names

['TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_1M',
 'TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_5M',
 'TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_15M',
 'TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_1H',
 'TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_6H',
 'TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_1D',
 'TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_7D',
 'TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_30D',
 'TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_1M',
 'TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_5M',
 'TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_15M',
 'TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_1H',
 'TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_6H',
 'TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_1D',
 'TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_7D',
 'TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_30D',
 'TAKENBY_CFMEDIADWELLTIMESIGNAL_1M',
 'TAKENBY_CFMEDIADWELLTIMESIGNAL_5M',
 'TAKENBY_CFMEDIADWELLTIMESIGNAL_15M',
 'TAKENBY_CFMEDIADWELLTIMESIGNAL_1H',
 'TAKENBY_CFMEDIADWELLTIMESIGNAL_6H',
 'TAKENBY_CFMEDIADWELLTIMESIGNAL_1D',
 'TAKENBY_CFMEDIADWELLTIMESIGNAL_7D',
 'TAKENBY_CFMEDIADWELLTIMESIGNAL_30D',
 'TAKENBY_CFMEDIATAPPEDTAK

In [37]:
js = '[{"TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_1M": NaN, "TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_5M": NaN, "TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_15M": NaN, "TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_1H": NaN, "TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_6H": NaN, "TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_1D": NaN, "TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_7D": NaN, "TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_30D": NaN, "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_1M": 7.696535994627496e-70, "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_5M": 2.618683775587088e-14, "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_15M": 4.7139070665777155e-05, "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_1H": 0.13935345825584178, "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_6H": 1.3060443652450713, "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_1D": 4.087081774077868, "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_7D": 24.170414981516267, "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_30D": 47.9210113498678, "TAKENBY_CFMEDIADWELLTIMESIGNAL_1M": 8.538857909291201e-05, "TAKENBY_CFMEDIADWELLTIMESIGNAL_5M": 0.3284413956793213, "TAKENBY_CFMEDIADWELLTIMESIGNAL_15M": 2.723490165222737, "TAKENBY_CFMEDIADWELLTIMESIGNAL_1H": 8.88357075285284, "TAKENBY_CFMEDIADWELLTIMESIGNAL_6H": 30.192759063305843, "TAKENBY_CFMEDIADWELLTIMESIGNAL_1D": 106.88837197367344, "TAKENBY_CFMEDIADWELLTIMESIGNAL_7D": 602.2156149638272, "TAKENBY_CFMEDIADWELLTIMESIGNAL_30D": 1069.405503367633, "TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_1M": 0.0, "TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_5M": 4.854684250953158e-205, "TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_15M": 7.859356634095807e-69, "TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_1H": 9.415572874205062e-18, "TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_6H": 0.0014539377113206632, "TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_1D": 0.23909054773795407, "TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_7D": 3.569064737928021, "TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_30D": 7.059845974899044, "TAKENBY_CFMEDIAVIEWEDSIGNAL_1M": 2.4838009675620147e-08, "TAKENBY_CFMEDIAVIEWEDSIGNAL_5M": 0.05793683725739394, "TAKENBY_CFMEDIAVIEWEDSIGNAL_15M": 1.0992148329222122, "TAKENBY_CFMEDIAVIEWEDSIGNAL_1H": 5.100071616785627, "TAKENBY_CFMEDIAVIEWEDSIGNAL_6H": 19.090003592037466, "TAKENBY_CFMEDIAVIEWEDSIGNAL_1D": 68.68856297720845, "TAKENBY_CFMEDIAVIEWEDSIGNAL_7D": 474.172794698875, "TAKENBY_CFMEDIAVIEWEDSIGNAL_30D": 997.7752196794028, "MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_1M": NaN, "MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_5M": NaN, "MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_15M": NaN, "MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_1H": NaN, "MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_6H": NaN, "MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_1D": NaN, "MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_7D": NaN, "MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_30D": NaN, "MEDIA_CFMEDIAADDEDREACTIONSIGNAL_1M": 7.696535994627496e-70, "MEDIA_CFMEDIAADDEDREACTIONSIGNAL_5M": 7.696535994627496e-70, "MEDIA_CFMEDIAADDEDREACTIONSIGNAL_15M": 7.696535994627496e-70, "MEDIA_CFMEDIAADDEDREACTIONSIGNAL_1H": 7.696535994627496e-70, "MEDIA_CFMEDIAADDEDREACTIONSIGNAL_6H": 7.696535994627496e-70, "MEDIA_CFMEDIAADDEDREACTIONSIGNAL_1D": 7.696535994627496e-70, "MEDIA_CFMEDIAADDEDREACTIONSIGNAL_7D": 7.696535994627496e-70, "MEDIA_CFMEDIAADDEDREACTIONSIGNAL_30D": 7.696535994627496e-70, "MEDIA_CFMEDIADWELLTIMESIGNAL_1M": 8.538857909291201e-05, "MEDIA_CFMEDIADWELLTIMESIGNAL_5M": 8.538857909291201e-05, "MEDIA_CFMEDIADWELLTIMESIGNAL_15M": 8.538857909291201e-05, "MEDIA_CFMEDIADWELLTIMESIGNAL_1H": 8.538857909291201e-05, "MEDIA_CFMEDIADWELLTIMESIGNAL_6H": 8.538857909291201e-05, "MEDIA_CFMEDIADWELLTIMESIGNAL_1D": 8.538857909291201e-05, "MEDIA_CFMEDIADWELLTIMESIGNAL_7D": 8.538857909291201e-05, "MEDIA_CFMEDIADWELLTIMESIGNAL_30D": 8.538857909291201e-05, "MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_1M": 4.2411915861046755e-243, "MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_5M": 4.2411915861046755e-243, "MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_15M": 4.2411915861046755e-243, "MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_1H": 4.2411915861046755e-243, "MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_6H": 4.2411915861046755e-243, "MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_1D": 4.2411915861046755e-243, "MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_7D": 4.2411915861046755e-243, "MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_30D": 4.2411915861046755e-243, "MEDIA_CFMEDIAVIEWEDSIGNAL_1M": 2.4838009675620147e-08, "MEDIA_CFMEDIAVIEWEDSIGNAL_5M": 2.4838009675620147e-08, "MEDIA_CFMEDIAVIEWEDSIGNAL_15M": 2.4838009675620147e-08, "MEDIA_CFMEDIAVIEWEDSIGNAL_1H": 2.4838009675620147e-08, "MEDIA_CFMEDIAVIEWEDSIGNAL_6H": 2.4838009675620147e-08, "MEDIA_CFMEDIAVIEWEDSIGNAL_1D": 2.4838009675620147e-08, "MEDIA_CFMEDIAVIEWEDSIGNAL_7D": 2.4838009675620147e-08, "MEDIA_CFMEDIAVIEWEDSIGNAL_30D": 2.4838009675620147e-08, "USER_CFMEDIAADDEDCOMMENTSIGNAL_1M": NaN, "USER_CFMEDIAADDEDCOMMENTSIGNAL_5M": NaN, "USER_CFMEDIAADDEDCOMMENTSIGNAL_15M": NaN, "USER_CFMEDIAADDEDCOMMENTSIGNAL_1H": NaN, "USER_CFMEDIAADDEDCOMMENTSIGNAL_6H": NaN, "USER_CFMEDIAADDEDCOMMENTSIGNAL_1D": NaN, "USER_CFMEDIAADDEDCOMMENTSIGNAL_7D": NaN, "USER_CFMEDIAADDEDCOMMENTSIGNAL_30D": NaN, "USER_CFMEDIAADDEDREACTIONSIGNAL_1M": NaN, "USER_CFMEDIAADDEDREACTIONSIGNAL_5M": NaN, "USER_CFMEDIAADDEDREACTIONSIGNAL_15M": NaN, "USER_CFMEDIAADDEDREACTIONSIGNAL_1H": NaN, "USER_CFMEDIAADDEDREACTIONSIGNAL_6H": NaN, "USER_CFMEDIAADDEDREACTIONSIGNAL_1D": NaN, "USER_CFMEDIAADDEDREACTIONSIGNAL_7D": NaN, "USER_CFMEDIAADDEDREACTIONSIGNAL_30D": NaN, "USER_CFMEDIADWELLTIMESIGNAL_1M": NaN, "USER_CFMEDIADWELLTIMESIGNAL_5M": NaN, "USER_CFMEDIADWELLTIMESIGNAL_15M": NaN, "USER_CFMEDIADWELLTIMESIGNAL_1H": NaN, "USER_CFMEDIADWELLTIMESIGNAL_6H": NaN, "USER_CFMEDIADWELLTIMESIGNAL_1D": NaN, "USER_CFMEDIADWELLTIMESIGNAL_7D": NaN, "USER_CFMEDIADWELLTIMESIGNAL_30D": NaN, "USER_CFMEDIATAPPEDTAKENBYSIGNAL_1M": NaN, "USER_CFMEDIATAPPEDTAKENBYSIGNAL_5M": NaN, "USER_CFMEDIATAPPEDTAKENBYSIGNAL_15M": NaN, "USER_CFMEDIATAPPEDTAKENBYSIGNAL_1H": NaN, "USER_CFMEDIATAPPEDTAKENBYSIGNAL_6H": NaN, "USER_CFMEDIATAPPEDTAKENBYSIGNAL_1D": NaN, "USER_CFMEDIATAPPEDTAKENBYSIGNAL_7D": NaN, "USER_CFMEDIATAPPEDTAKENBYSIGNAL_30D": NaN, "USER_CFMEDIAVIEWEDSIGNAL_1M": NaN, "USER_CFMEDIAVIEWEDSIGNAL_5M": NaN, "USER_CFMEDIAVIEWEDSIGNAL_15M": NaN, "USER_CFMEDIAVIEWEDSIGNAL_1H": NaN, "USER_CFMEDIAVIEWEDSIGNAL_6H": NaN, "USER_CFMEDIAVIEWEDSIGNAL_1D": NaN, "USER_CFMEDIAVIEWEDSIGNAL_7D": NaN, "USER_CFMEDIAVIEWEDSIGNAL_30D": NaN, "CROSS_CFMEDIAADDEDCOMMENTSIGNAL_1M": NaN, "CROSS_CFMEDIAADDEDCOMMENTSIGNAL_5M": NaN, "CROSS_CFMEDIAADDEDCOMMENTSIGNAL_15M": NaN, "CROSS_CFMEDIAADDEDCOMMENTSIGNAL_1H": NaN, "CROSS_CFMEDIAADDEDCOMMENTSIGNAL_6H": NaN, "CROSS_CFMEDIAADDEDCOMMENTSIGNAL_1D": NaN, "CROSS_CFMEDIAADDEDCOMMENTSIGNAL_7D": NaN, "CROSS_CFMEDIAADDEDCOMMENTSIGNAL_30D": NaN, "CROSS_CFMEDIAADDEDREACTIONSIGNAL_1M": NaN, "CROSS_CFMEDIAADDEDREACTIONSIGNAL_5M": NaN, "CROSS_CFMEDIAADDEDREACTIONSIGNAL_15M": NaN, "CROSS_CFMEDIAADDEDREACTIONSIGNAL_1H": NaN, "CROSS_CFMEDIAADDEDREACTIONSIGNAL_6H": NaN, "CROSS_CFMEDIAADDEDREACTIONSIGNAL_1D": NaN, "CROSS_CFMEDIAADDEDREACTIONSIGNAL_7D": NaN, "CROSS_CFMEDIAADDEDREACTIONSIGNAL_30D": NaN, "CROSS_CFMEDIADWELLTIMESIGNAL_1M": NaN, "CROSS_CFMEDIADWELLTIMESIGNAL_5M": NaN, "CROSS_CFMEDIADWELLTIMESIGNAL_15M": NaN, "CROSS_CFMEDIADWELLTIMESIGNAL_1H": NaN, "CROSS_CFMEDIADWELLTIMESIGNAL_6H": NaN, "CROSS_CFMEDIADWELLTIMESIGNAL_1D": NaN, "CROSS_CFMEDIADWELLTIMESIGNAL_7D": NaN, "CROSS_CFMEDIADWELLTIMESIGNAL_30D": NaN, "CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_1M": NaN, "CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_5M": NaN, "CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_15M": NaN, "CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_1H": NaN, "CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_6H": NaN, "CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_1D": NaN, "CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_7D": NaN, "CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_30D": NaN, "CROSS_CFMEDIAVIEWEDSIGNAL_1M": NaN, "CROSS_CFMEDIAVIEWEDSIGNAL_5M": NaN, "CROSS_CFMEDIAVIEWEDSIGNAL_15M": NaN, "CROSS_CFMEDIAVIEWEDSIGNAL_1H": NaN, "CROSS_CFMEDIAVIEWEDSIGNAL_6H": NaN, "CROSS_CFMEDIAVIEWEDSIGNAL_1D": NaN, "CROSS_CFMEDIAVIEWEDSIGNAL_7D": NaN, "CROSS_CFMEDIAVIEWEDSIGNAL_30D": NaN}, {"TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_1M": 0.0, "TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_5M": 0.0, "TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_15M": 0.0, "TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_1H": 1.020605472667207e-83, "TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_6H": 1.4729179292826815e-14, "TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_1D": 0.00039939363005160426, "TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_7D": 1.0526957220514856, "TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_30D": 3.4493179034603187, "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_1M": 3.505111552056011e-79, "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_5M": 2.0367623991528018e-16, "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_15M": 5.8836494528536475e-06, "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_1H": 0.04926916276901351, "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_6H": 0.922022135754758, "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_1D": 4.9573878897004935, "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_7D": 61.40894849032628, "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_30D": 133.9274308177213, "TAKENBY_CFMEDIADWELLTIMESIGNAL_1M": 4.6567391977682815e-08, "TAKENBY_CFMEDIADWELLTIMESIGNAL_5M": 0.029530452205541655, "TAKENBY_CFMEDIADWELLTIMESIGNAL_15M": 0.30055009286429, "TAKENBY_CFMEDIADWELLTIMESIGNAL_1H": 3.239780273774478, "TAKENBY_CFMEDIADWELLTIMESIGNAL_6H": 37.47047354146888, "TAKENBY_CFMEDIADWELLTIMESIGNAL_1D": 209.5427510265734, "TAKENBY_CFMEDIADWELLTIMESIGNAL_7D": 1359.982186995942, "TAKENBY_CFMEDIADWELLTIMESIGNAL_30D": 2366.9015260642345, "TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_1M": 0.0, "TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_5M": 1.909956210035835e-93, "TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_15M": 1.2407219629470397e-31, "TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_1H": 1.8769249387889326e-08, "TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_6H": 0.06471222112564023, "TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_1D": 1.0791410195359823, "TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_7D": 8.241592283931768, "TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_30D": 16.371188826051238, "TAKENBY_CFMEDIAVIEWEDSIGNAL_1M": 1.5943032544813673e-05, "TAKENBY_CFMEDIAVIEWEDSIGNAL_5M": 0.14513801267112597, "TAKENBY_CFMEDIAVIEWEDSIGNAL_15M": 0.8186275470119576, "TAKENBY_CFMEDIAVIEWEDSIGNAL_1H": 2.962743499219805, "TAKENBY_CFMEDIAVIEWEDSIGNAL_6H": 19.607729456069396, "TAKENBY_CFMEDIAVIEWEDSIGNAL_1D": 73.20150250603649, "TAKENBY_CFMEDIAVIEWEDSIGNAL_7D": 490.69008713307136, "TAKENBY_CFMEDIAVIEWEDSIGNAL_30D": 986.5911741866066, "MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_1M": 0.0, "MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_5M": 0.0, "MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_15M": 0.0, "MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_1H": 0.0, "MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_6H": 0.0, "MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_1D": 0.0, "MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_7D": 0.0, "MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_30D": 0.0, "MEDIA_CFMEDIAADDEDREACTIONSIGNAL_1M": 3.505111552056011e-79, "MEDIA_CFMEDIAADDEDREACTIONSIGNAL_5M": 3.505111552056011e-79, "MEDIA_CFMEDIAADDEDREACTIONSIGNAL_15M": 3.505111552056011e-79, "MEDIA_CFMEDIAADDEDREACTIONSIGNAL_1H": 3.505111552056011e-79, "MEDIA_CFMEDIAADDEDREACTIONSIGNAL_6H": 3.505111552056011e-79, "MEDIA_CFMEDIAADDEDREACTIONSIGNAL_1D": 3.505111552056011e-79, "MEDIA_CFMEDIAADDEDREACTIONSIGNAL_7D": 3.505111552056011e-79, "MEDIA_CFMEDIAADDEDREACTIONSIGNAL_30D": 3.505111552056011e-79, "MEDIA_CFMEDIADWELLTIMESIGNAL_1M": 4.6567391977682815e-08, "MEDIA_CFMEDIADWELLTIMESIGNAL_5M": 4.6567391977682815e-08, "MEDIA_CFMEDIADWELLTIMESIGNAL_15M": 4.6567391977682815e-08, "MEDIA_CFMEDIADWELLTIMESIGNAL_1H": 4.6567391977682815e-08, "MEDIA_CFMEDIADWELLTIMESIGNAL_6H": 4.6567391977682815e-08, "MEDIA_CFMEDIADWELLTIMESIGNAL_1D": 4.6567391977682815e-08, "MEDIA_CFMEDIADWELLTIMESIGNAL_7D": 4.6567391977682815e-08, "MEDIA_CFMEDIADWELLTIMESIGNAL_30D": 4.6567391977682815e-08, "MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_1M": 0.0, "MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_5M": 0.0, "MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_15M": 0.0, "MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_1H": 0.0, "MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_6H": 0.0, "MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_1D": 0.0, "MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_7D": 0.0, "MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_30D": 0.0, "MEDIA_CFMEDIAVIEWEDSIGNAL_1M": 1.5943032544813673e-05, "MEDIA_CFMEDIAVIEWEDSIGNAL_5M": 1.5943032544813673e-05, "MEDIA_CFMEDIAVIEWEDSIGNAL_15M": 1.5943032544813673e-05, "MEDIA_CFMEDIAVIEWEDSIGNAL_1H": 1.5943032544813673e-05, "MEDIA_CFMEDIAVIEWEDSIGNAL_6H": 1.5943032544813673e-05, "MEDIA_CFMEDIAVIEWEDSIGNAL_1D": 1.5943032544813673e-05, "MEDIA_CFMEDIAVIEWEDSIGNAL_7D": 1.5943032544813673e-05, "MEDIA_CFMEDIAVIEWEDSIGNAL_30D": 1.5943032544813673e-05, "USER_CFMEDIAADDEDCOMMENTSIGNAL_1M": NaN, "USER_CFMEDIAADDEDCOMMENTSIGNAL_5M": NaN, "USER_CFMEDIAADDEDCOMMENTSIGNAL_15M": NaN, "USER_CFMEDIAADDEDCOMMENTSIGNAL_1H": NaN, "USER_CFMEDIAADDEDCOMMENTSIGNAL_6H": NaN, "USER_CFMEDIAADDEDCOMMENTSIGNAL_1D": NaN, "USER_CFMEDIAADDEDCOMMENTSIGNAL_7D": NaN, "USER_CFMEDIAADDEDCOMMENTSIGNAL_30D": NaN, "USER_CFMEDIAADDEDREACTIONSIGNAL_1M": NaN, "USER_CFMEDIAADDEDREACTIONSIGNAL_5M": NaN, "USER_CFMEDIAADDEDREACTIONSIGNAL_15M": NaN, "USER_CFMEDIAADDEDREACTIONSIGNAL_1H": NaN, "USER_CFMEDIAADDEDREACTIONSIGNAL_6H": NaN, "USER_CFMEDIAADDEDREACTIONSIGNAL_1D": NaN, "USER_CFMEDIAADDEDREACTIONSIGNAL_7D": NaN, "USER_CFMEDIAADDEDREACTIONSIGNAL_30D": NaN, "USER_CFMEDIADWELLTIMESIGNAL_1M": NaN, "USER_CFMEDIADWELLTIMESIGNAL_5M": NaN, "USER_CFMEDIADWELLTIMESIGNAL_15M": NaN, "USER_CFMEDIADWELLTIMESIGNAL_1H": NaN, "USER_CFMEDIADWELLTIMESIGNAL_6H": NaN, "USER_CFMEDIADWELLTIMESIGNAL_1D": NaN, "USER_CFMEDIADWELLTIMESIGNAL_7D": NaN, "USER_CFMEDIADWELLTIMESIGNAL_30D": NaN, "USER_CFMEDIATAPPEDTAKENBYSIGNAL_1M": NaN, "USER_CFMEDIATAPPEDTAKENBYSIGNAL_5M": NaN, "USER_CFMEDIATAPPEDTAKENBYSIGNAL_15M": NaN, "USER_CFMEDIATAPPEDTAKENBYSIGNAL_1H": NaN, "USER_CFMEDIATAPPEDTAKENBYSIGNAL_6H": NaN, "USER_CFMEDIATAPPEDTAKENBYSIGNAL_1D": NaN, "USER_CFMEDIATAPPEDTAKENBYSIGNAL_7D": NaN, "USER_CFMEDIATAPPEDTAKENBYSIGNAL_30D": NaN, "USER_CFMEDIAVIEWEDSIGNAL_1M": NaN, "USER_CFMEDIAVIEWEDSIGNAL_5M": NaN, "USER_CFMEDIAVIEWEDSIGNAL_15M": NaN, "USER_CFMEDIAVIEWEDSIGNAL_1H": NaN, "USER_CFMEDIAVIEWEDSIGNAL_6H": NaN, "USER_CFMEDIAVIEWEDSIGNAL_1D": NaN, "USER_CFMEDIAVIEWEDSIGNAL_7D": NaN, "USER_CFMEDIAVIEWEDSIGNAL_30D": NaN, "CROSS_CFMEDIAADDEDCOMMENTSIGNAL_1M": NaN, "CROSS_CFMEDIAADDEDCOMMENTSIGNAL_5M": NaN, "CROSS_CFMEDIAADDEDCOMMENTSIGNAL_15M": NaN, "CROSS_CFMEDIAADDEDCOMMENTSIGNAL_1H": NaN, "CROSS_CFMEDIAADDEDCOMMENTSIGNAL_6H": NaN, "CROSS_CFMEDIAADDEDCOMMENTSIGNAL_1D": NaN, "CROSS_CFMEDIAADDEDCOMMENTSIGNAL_7D": NaN, "CROSS_CFMEDIAADDEDCOMMENTSIGNAL_30D": NaN, "CROSS_CFMEDIAADDEDREACTIONSIGNAL_1M": NaN, "CROSS_CFMEDIAADDEDREACTIONSIGNAL_5M": NaN, "CROSS_CFMEDIAADDEDREACTIONSIGNAL_15M": NaN, "CROSS_CFMEDIAADDEDREACTIONSIGNAL_1H": NaN, "CROSS_CFMEDIAADDEDREACTIONSIGNAL_6H": NaN, "CROSS_CFMEDIAADDEDREACTIONSIGNAL_1D": NaN, "CROSS_CFMEDIAADDEDREACTIONSIGNAL_7D": NaN, "CROSS_CFMEDIAADDEDREACTIONSIGNAL_30D": NaN, "CROSS_CFMEDIADWELLTIMESIGNAL_1M": NaN, "CROSS_CFMEDIADWELLTIMESIGNAL_5M": NaN, "CROSS_CFMEDIADWELLTIMESIGNAL_15M": NaN, "CROSS_CFMEDIADWELLTIMESIGNAL_1H": NaN, "CROSS_CFMEDIADWELLTIMESIGNAL_6H": NaN, "CROSS_CFMEDIADWELLTIMESIGNAL_1D": NaN, "CROSS_CFMEDIADWELLTIMESIGNAL_7D": NaN, "CROSS_CFMEDIADWELLTIMESIGNAL_30D": NaN, "CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_1M": NaN, "CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_5M": NaN, "CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_15M": NaN, "CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_1H": NaN, "CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_6H": NaN, "CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_1D": NaN, "CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_7D": NaN, "CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_30D": NaN, "CROSS_CFMEDIAVIEWEDSIGNAL_1M": NaN, "CROSS_CFMEDIAVIEWEDSIGNAL_5M": NaN, "CROSS_CFMEDIAVIEWEDSIGNAL_15M": NaN, "CROSS_CFMEDIAVIEWEDSIGNAL_1H": NaN, "CROSS_CFMEDIAVIEWEDSIGNAL_6H": NaN, "CROSS_CFMEDIAVIEWEDSIGNAL_1D": NaN, "CROSS_CFMEDIAVIEWEDSIGNAL_7D": NaN, "CROSS_CFMEDIAVIEWEDSIGNAL_30D": NaN}]'
js = '[{"TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_1M": NaN, "TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_5M": NaN, "TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_15M": NaN, "TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_1H": NaN, "TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_6H": NaN, "TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_1D": NaN, "TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_7D": NaN, "TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_30D": NaN, "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_1M": 7.696535994627496e-70, "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_5M": 2.618683775587088e-14, "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_15M": 4.7139070665777155e-05, "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_1H": 0.13935345825584178, "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_6H": 1.3060443652450713, "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_1D": 4.087081774077868, "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_7D": 24.170414981516267, "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_30D": 47.9210113498678, "TAKENBY_CFMEDIADWELLTIMESIGNAL_1M": 8.538857909291201e-05, "TAKENBY_CFMEDIADWELLTIMESIGNAL_5M": 0.3284413956793213, "TAKENBY_CFMEDIADWELLTIMESIGNAL_15M": 2.723490165222737, "TAKENBY_CFMEDIADWELLTIMESIGNAL_1H": 8.88357075285284, "TAKENBY_CFMEDIADWELLTIMESIGNAL_6H": 30.192759063305843, "TAKENBY_CFMEDIADWELLTIMESIGNAL_1D": 106.88837197367344, "TAKENBY_CFMEDIADWELLTIMESIGNAL_7D": 602.2156149638272, "TAKENBY_CFMEDIADWELLTIMESIGNAL_30D": 1069.405503367633, "TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_1M": 0.0, "TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_5M": 4.854684250953158e-205, "TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_15M": 7.859356634095807e-69, "TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_1H": 9.415572874205062e-18, "TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_6H": 0.0014539377113206632, "TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_1D": 0.23909054773795407, "TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_7D": 3.569064737928021, "TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_30D": 7.059845974899044, "TAKENBY_CFMEDIAVIEWEDSIGNAL_1M": 2.4838009675620147e-08, "TAKENBY_CFMEDIAVIEWEDSIGNAL_5M": 0.05793683725739394, "TAKENBY_CFMEDIAVIEWEDSIGNAL_15M": 1.0992148329222122, "TAKENBY_CFMEDIAVIEWEDSIGNAL_1H": 5.100071616785627, "TAKENBY_CFMEDIAVIEWEDSIGNAL_6H": 19.090003592037466, "TAKENBY_CFMEDIAVIEWEDSIGNAL_1D": 68.68856297720845, "TAKENBY_CFMEDIAVIEWEDSIGNAL_7D": 474.172794698875, "TAKENBY_CFMEDIAVIEWEDSIGNAL_30D": 997.7752196794028, "MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_1M": NaN, "MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_5M": NaN, "MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_15M": NaN, "MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_1H": NaN, "MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_6H": NaN, "MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_1D": NaN, "MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_7D": NaN, "MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_30D": NaN, "MEDIA_CFMEDIAADDEDREACTIONSIGNAL_1M": 7.696535994627496e-70, "MEDIA_CFMEDIAADDEDREACTIONSIGNAL_5M": 7.696535994627496e-70, "MEDIA_CFMEDIAADDEDREACTIONSIGNAL_15M": 7.696535994627496e-70, "MEDIA_CFMEDIAADDEDREACTIONSIGNAL_1H": 7.696535994627496e-70, "MEDIA_CFMEDIAADDEDREACTIONSIGNAL_6H": 7.696535994627496e-70, "MEDIA_CFMEDIAADDEDREACTIONSIGNAL_1D": 7.696535994627496e-70, "MEDIA_CFMEDIAADDEDREACTIONSIGNAL_7D": 7.696535994627496e-70, "MEDIA_CFMEDIAADDEDREACTIONSIGNAL_30D": 7.696535994627496e-70, "MEDIA_CFMEDIADWELLTIMESIGNAL_1M": 8.538857909291201e-05, "MEDIA_CFMEDIADWELLTIMESIGNAL_5M": 8.538857909291201e-05, "MEDIA_CFMEDIADWELLTIMESIGNAL_15M": 8.538857909291201e-05, "MEDIA_CFMEDIADWELLTIMESIGNAL_1H": 8.538857909291201e-05, "MEDIA_CFMEDIADWELLTIMESIGNAL_6H": 8.538857909291201e-05, "MEDIA_CFMEDIADWELLTIMESIGNAL_1D": 8.538857909291201e-05, "MEDIA_CFMEDIADWELLTIMESIGNAL_7D": 8.538857909291201e-05, "MEDIA_CFMEDIADWELLTIMESIGNAL_30D": 8.538857909291201e-05, "MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_1M": 4.2411915861046755e-243, "MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_5M": 4.2411915861046755e-243, "MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_15M": 4.2411915861046755e-243, "MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_1H": 4.2411915861046755e-243, "MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_6H": 4.2411915861046755e-243, "MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_1D": 4.2411915861046755e-243, "MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_7D": 4.2411915861046755e-243, "MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_30D": 4.2411915861046755e-243, "MEDIA_CFMEDIAVIEWEDSIGNAL_1M": 2.4838009675620147e-08, "MEDIA_CFMEDIAVIEWEDSIGNAL_5M": 2.4838009675620147e-08, "MEDIA_CFMEDIAVIEWEDSIGNAL_15M": 2.4838009675620147e-08, "MEDIA_CFMEDIAVIEWEDSIGNAL_1H": 2.4838009675620147e-08, "MEDIA_CFMEDIAVIEWEDSIGNAL_6H": 2.4838009675620147e-08, "MEDIA_CFMEDIAVIEWEDSIGNAL_1D": 2.4838009675620147e-08, "MEDIA_CFMEDIAVIEWEDSIGNAL_7D": 2.4838009675620147e-08, "MEDIA_CFMEDIAVIEWEDSIGNAL_30D": 2.4838009675620147e-08, "USER_CFMEDIAADDEDCOMMENTSIGNAL_1M": NaN, "USER_CFMEDIAADDEDCOMMENTSIGNAL_5M": NaN, "USER_CFMEDIAADDEDCOMMENTSIGNAL_15M": NaN, "USER_CFMEDIAADDEDCOMMENTSIGNAL_1H": NaN, "USER_CFMEDIAADDEDCOMMENTSIGNAL_6H": NaN, "USER_CFMEDIAADDEDCOMMENTSIGNAL_1D": NaN, "USER_CFMEDIAADDEDCOMMENTSIGNAL_7D": NaN, "USER_CFMEDIAADDEDCOMMENTSIGNAL_30D": NaN, "USER_CFMEDIAADDEDREACTIONSIGNAL_1M": NaN, "USER_CFMEDIAADDEDREACTIONSIGNAL_5M": NaN, "USER_CFMEDIAADDEDREACTIONSIGNAL_15M": NaN, "USER_CFMEDIAADDEDREACTIONSIGNAL_1H": NaN, "USER_CFMEDIAADDEDREACTIONSIGNAL_6H": NaN, "USER_CFMEDIAADDEDREACTIONSIGNAL_1D": NaN, "USER_CFMEDIAADDEDREACTIONSIGNAL_7D": NaN, "USER_CFMEDIAADDEDREACTIONSIGNAL_30D": NaN, "USER_CFMEDIADWELLTIMESIGNAL_1M": NaN, "USER_CFMEDIADWELLTIMESIGNAL_5M": NaN, "USER_CFMEDIADWELLTIMESIGNAL_15M": NaN, "USER_CFMEDIADWELLTIMESIGNAL_1H": NaN, "USER_CFMEDIADWELLTIMESIGNAL_6H": NaN, "USER_CFMEDIADWELLTIMESIGNAL_1D": NaN, "USER_CFMEDIADWELLTIMESIGNAL_7D": NaN, "USER_CFMEDIADWELLTIMESIGNAL_30D": NaN, "USER_CFMEDIATAPPEDTAKENBYSIGNAL_1M": NaN, "USER_CFMEDIATAPPEDTAKENBYSIGNAL_5M": NaN, "USER_CFMEDIATAPPEDTAKENBYSIGNAL_15M": NaN, "USER_CFMEDIATAPPEDTAKENBYSIGNAL_1H": NaN, "USER_CFMEDIATAPPEDTAKENBYSIGNAL_6H": NaN, "USER_CFMEDIATAPPEDTAKENBYSIGNAL_1D": NaN, "USER_CFMEDIATAPPEDTAKENBYSIGNAL_7D": NaN, "USER_CFMEDIATAPPEDTAKENBYSIGNAL_30D": NaN, "USER_CFMEDIAVIEWEDSIGNAL_1M": NaN, "USER_CFMEDIAVIEWEDSIGNAL_5M": NaN, "USER_CFMEDIAVIEWEDSIGNAL_15M": NaN, "USER_CFMEDIAVIEWEDSIGNAL_1H": NaN, "USER_CFMEDIAVIEWEDSIGNAL_6H": NaN, "USER_CFMEDIAVIEWEDSIGNAL_1D": NaN, "USER_CFMEDIAVIEWEDSIGNAL_7D": NaN, "USER_CFMEDIAVIEWEDSIGNAL_30D": NaN, "CROSS_CFMEDIAADDEDCOMMENTSIGNAL_1M": NaN, "CROSS_CFMEDIAADDEDCOMMENTSIGNAL_5M": NaN, "CROSS_CFMEDIAADDEDCOMMENTSIGNAL_15M": NaN, "CROSS_CFMEDIAADDEDCOMMENTSIGNAL_1H": NaN, "CROSS_CFMEDIAADDEDCOMMENTSIGNAL_6H": NaN, "CROSS_CFMEDIAADDEDCOMMENTSIGNAL_1D": NaN, "CROSS_CFMEDIAADDEDCOMMENTSIGNAL_7D": NaN, "CROSS_CFMEDIAADDEDCOMMENTSIGNAL_30D": NaN, "CROSS_CFMEDIAADDEDREACTIONSIGNAL_1M": NaN, "CROSS_CFMEDIAADDEDREACTIONSIGNAL_5M": NaN, "CROSS_CFMEDIAADDEDREACTIONSIGNAL_15M": NaN, "CROSS_CFMEDIAADDEDREACTIONSIGNAL_1H": NaN, "CROSS_CFMEDIAADDEDREACTIONSIGNAL_6H": NaN, "CROSS_CFMEDIAADDEDREACTIONSIGNAL_1D": NaN, "CROSS_CFMEDIAADDEDREACTIONSIGNAL_7D": NaN, "CROSS_CFMEDIAADDEDREACTIONSIGNAL_30D": NaN, "CROSS_CFMEDIADWELLTIMESIGNAL_1M": NaN, "CROSS_CFMEDIADWELLTIMESIGNAL_5M": NaN, "CROSS_CFMEDIADWELLTIMESIGNAL_15M": NaN, "CROSS_CFMEDIADWELLTIMESIGNAL_1H": NaN, "CROSS_CFMEDIADWELLTIMESIGNAL_6H": NaN, "CROSS_CFMEDIADWELLTIMESIGNAL_1D": NaN, "CROSS_CFMEDIADWELLTIMESIGNAL_7D": NaN, "CROSS_CFMEDIADWELLTIMESIGNAL_30D": NaN, "CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_1M": NaN, "CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_5M": NaN, "CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_15M": NaN, "CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_1H": NaN, "CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_6H": NaN, "CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_1D": NaN, "CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_7D": NaN, "CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_30D": NaN, "CROSS_CFMEDIAVIEWEDSIGNAL_1M": NaN, "CROSS_CFMEDIAVIEWEDSIGNAL_5M": NaN, "CROSS_CFMEDIAVIEWEDSIGNAL_15M": NaN, "CROSS_CFMEDIAVIEWEDSIGNAL_1H": NaN, "CROSS_CFMEDIAVIEWEDSIGNAL_6H": NaN, "CROSS_CFMEDIAVIEWEDSIGNAL_1D": NaN, "CROSS_CFMEDIAVIEWEDSIGNAL_7D": NaN, "CROSS_CFMEDIAVIEWEDSIGNAL_30D": NaN}, {"TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_1M": 0.0, "TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_5M": 0.0, "TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_15M": 0.0, "TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_1H": 1.020605472667207e-83, "TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_6H": 1.4729179292826815e-14, "TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_1D": 0.00039939363005160426, "TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_7D": 1.0526957220514856, "TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_30D": 3.4493179034603187, "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_1M": 3.505111552056011e-79, "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_5M": 2.0367623991528018e-16, "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_15M": 5.8836494528536475e-06, "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_1H": 0.04926916276901351, "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_6H": 0.922022135754758, "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_1D": 4.9573878897004935, "TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_7D": 61.40894849032628, "TAKENBY_CFMEDIADWELLTIMESIGNAL_1M": 4.6567391977682815e-08, "TAKENBY_CFMEDIADWELLTIMESIGNAL_5M": 0.029530452205541655, "TAKENBY_CFMEDIADWELLTIMESIGNAL_15M": 0.30055009286429, "TAKENBY_CFMEDIADWELLTIMESIGNAL_1H": 3.239780273774478, "TAKENBY_CFMEDIADWELLTIMESIGNAL_6H": 37.47047354146888, "TAKENBY_CFMEDIADWELLTIMESIGNAL_1D": 209.5427510265734, "TAKENBY_CFMEDIADWELLTIMESIGNAL_7D": 1359.982186995942, "TAKENBY_CFMEDIADWELLTIMESIGNAL_30D": 2366.9015260642345, "TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_1M": 0.0, "TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_5M": 1.909956210035835e-93, "TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_15M": 1.2407219629470397e-31, "TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_1H": 1.8769249387889326e-08, "TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_6H": 0.06471222112564023, "TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_1D": 1.0791410195359823, "TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_7D": 8.241592283931768, "TAKENBY_CFMEDIATAPPEDTAKENBYSIGNAL_30D": 16.371188826051238, "TAKENBY_CFMEDIAVIEWEDSIGNAL_1M": 1.5943032544813673e-05, "TAKENBY_CFMEDIAVIEWEDSIGNAL_5M": 0.14513801267112597, "TAKENBY_CFMEDIAVIEWEDSIGNAL_15M": 0.8186275470119576, "TAKENBY_CFMEDIAVIEWEDSIGNAL_1H": 2.962743499219805, "TAKENBY_CFMEDIAVIEWEDSIGNAL_6H": 19.607729456069396, "TAKENBY_CFMEDIAVIEWEDSIGNAL_1D": 73.20150250603649, "TAKENBY_CFMEDIAVIEWEDSIGNAL_7D": 490.69008713307136, "TAKENBY_CFMEDIAVIEWEDSIGNAL_30D": 986.5911741866066, "MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_1M": 0.0, "MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_5M": 0.0, "MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_15M": 0.0, "MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_1H": 0.0, "MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_6H": 0.0, "MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_1D": 0.0, "MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_7D": 0.0, "MEDIA_CFMEDIAADDEDCOMMENTSIGNAL_30D": 0.0, "MEDIA_CFMEDIAADDEDREACTIONSIGNAL_1M": 3.505111552056011e-79, "MEDIA_CFMEDIAADDEDREACTIONSIGNAL_5M": 3.505111552056011e-79, "MEDIA_CFMEDIAADDEDREACTIONSIGNAL_15M": 3.505111552056011e-79, "MEDIA_CFMEDIAADDEDREACTIONSIGNAL_1H": 3.505111552056011e-79, "MEDIA_CFMEDIAADDEDREACTIONSIGNAL_6H": 3.505111552056011e-79, "MEDIA_CFMEDIAADDEDREACTIONSIGNAL_1D": 3.505111552056011e-79, "MEDIA_CFMEDIAADDEDREACTIONSIGNAL_7D": 3.505111552056011e-79, "MEDIA_CFMEDIAADDEDREACTIONSIGNAL_30D": 3.505111552056011e-79, "MEDIA_CFMEDIADWELLTIMESIGNAL_1M": 4.6567391977682815e-08, "MEDIA_CFMEDIADWELLTIMESIGNAL_5M": 4.6567391977682815e-08, "MEDIA_CFMEDIADWELLTIMESIGNAL_15M": 4.6567391977682815e-08, "MEDIA_CFMEDIADWELLTIMESIGNAL_1H": 4.6567391977682815e-08, "MEDIA_CFMEDIADWELLTIMESIGNAL_6H": 4.6567391977682815e-08, "MEDIA_CFMEDIADWELLTIMESIGNAL_7D": 4.6567391977682815e-08, "MEDIA_CFMEDIADWELLTIMESIGNAL_30D": 4.6567391977682815e-08, "MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_1M": 0.0, "MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_5M": 0.0, "MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_15M": 0.0, "MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_1H": 0.0, "MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_6H": 0.0, "MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_1D": 0.0, "MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_7D": 0.0, "MEDIA_CFMEDIATAPPEDTAKENBYSIGNAL_30D": 0.0, "MEDIA_CFMEDIAVIEWEDSIGNAL_1M": 1.5943032544813673e-05, "MEDIA_CFMEDIAVIEWEDSIGNAL_5M": 1.5943032544813673e-05, "MEDIA_CFMEDIAVIEWEDSIGNAL_15M": 1.5943032544813673e-05, "MEDIA_CFMEDIAVIEWEDSIGNAL_1H": 1.5943032544813673e-05, "MEDIA_CFMEDIAVIEWEDSIGNAL_6H": 1.5943032544813673e-05, "MEDIA_CFMEDIAVIEWEDSIGNAL_1D": 1.5943032544813673e-05, "MEDIA_CFMEDIAVIEWEDSIGNAL_7D": 1.5943032544813673e-05, "MEDIA_CFMEDIAVIEWEDSIGNAL_30D": 1.5943032544813673e-05, "USER_CFMEDIAADDEDCOMMENTSIGNAL_1M": NaN, "USER_CFMEDIAADDEDCOMMENTSIGNAL_5M": NaN, "USER_CFMEDIAADDEDCOMMENTSIGNAL_15M": NaN, "USER_CFMEDIAADDEDCOMMENTSIGNAL_1H": NaN, "USER_CFMEDIAADDEDCOMMENTSIGNAL_6H": NaN, "USER_CFMEDIAADDEDCOMMENTSIGNAL_1D": NaN, "USER_CFMEDIAADDEDCOMMENTSIGNAL_7D": NaN, "USER_CFMEDIAADDEDCOMMENTSIGNAL_30D": NaN, "USER_CFMEDIAADDEDREACTIONSIGNAL_1M": NaN, "USER_CFMEDIAADDEDREACTIONSIGNAL_5M": NaN, "USER_CFMEDIAADDEDREACTIONSIGNAL_15M": NaN, "USER_CFMEDIAADDEDREACTIONSIGNAL_1H": NaN, "USER_CFMEDIAADDEDREACTIONSIGNAL_6H": NaN, "USER_CFMEDIAADDEDREACTIONSIGNAL_1D": NaN, "USER_CFMEDIAADDEDREACTIONSIGNAL_7D": NaN, "USER_CFMEDIAADDEDREACTIONSIGNAL_30D": NaN, "USER_CFMEDIADWELLTIMESIGNAL_1M": NaN, "USER_CFMEDIADWELLTIMESIGNAL_5M": NaN, "USER_CFMEDIADWELLTIMESIGNAL_15M": NaN, "USER_CFMEDIADWELLTIMESIGNAL_1H": NaN, "USER_CFMEDIADWELLTIMESIGNAL_6H": NaN, "USER_CFMEDIADWELLTIMESIGNAL_1D": NaN, "USER_CFMEDIADWELLTIMESIGNAL_7D": NaN, "USER_CFMEDIADWELLTIMESIGNAL_30D": NaN, "USER_CFMEDIATAPPEDTAKENBYSIGNAL_1M": NaN, "USER_CFMEDIATAPPEDTAKENBYSIGNAL_5M": NaN, "USER_CFMEDIATAPPEDTAKENBYSIGNAL_15M": NaN, "USER_CFMEDIATAPPEDTAKENBYSIGNAL_1H": NaN, "USER_CFMEDIATAPPEDTAKENBYSIGNAL_6H": NaN, "USER_CFMEDIATAPPEDTAKENBYSIGNAL_1D": NaN, "USER_CFMEDIATAPPEDTAKENBYSIGNAL_7D": NaN, "USER_CFMEDIATAPPEDTAKENBYSIGNAL_30D": NaN, "USER_CFMEDIAVIEWEDSIGNAL_1M": NaN, "USER_CFMEDIAVIEWEDSIGNAL_5M": NaN, "USER_CFMEDIAVIEWEDSIGNAL_15M": NaN, "USER_CFMEDIAVIEWEDSIGNAL_1H": NaN, "USER_CFMEDIAVIEWEDSIGNAL_6H": NaN, "USER_CFMEDIAVIEWEDSIGNAL_1D": NaN, "USER_CFMEDIAVIEWEDSIGNAL_7D": NaN, "USER_CFMEDIAVIEWEDSIGNAL_30D": NaN, "CROSS_CFMEDIAADDEDCOMMENTSIGNAL_1M": NaN, "CROSS_CFMEDIAADDEDCOMMENTSIGNAL_5M": NaN, "CROSS_CFMEDIAADDEDCOMMENTSIGNAL_15M": NaN, "CROSS_CFMEDIAADDEDCOMMENTSIGNAL_1H": NaN, "CROSS_CFMEDIAADDEDCOMMENTSIGNAL_6H": NaN, "CROSS_CFMEDIAADDEDCOMMENTSIGNAL_1D": NaN, "CROSS_CFMEDIAADDEDCOMMENTSIGNAL_7D": NaN, "CROSS_CFMEDIAADDEDCOMMENTSIGNAL_30D": NaN, "CROSS_CFMEDIAADDEDREACTIONSIGNAL_1M": NaN, "CROSS_CFMEDIAADDEDREACTIONSIGNAL_5M": NaN, "CROSS_CFMEDIAADDEDREACTIONSIGNAL_15M": NaN, "CROSS_CFMEDIAADDEDREACTIONSIGNAL_1H": NaN, "CROSS_CFMEDIAADDEDREACTIONSIGNAL_6H": NaN, "CROSS_CFMEDIAADDEDREACTIONSIGNAL_1D": NaN, "CROSS_CFMEDIAADDEDREACTIONSIGNAL_7D": NaN, "CROSS_CFMEDIAADDEDREACTIONSIGNAL_30D": NaN, "CROSS_CFMEDIADWELLTIMESIGNAL_1M": NaN, "CROSS_CFMEDIADWELLTIMESIGNAL_5M": NaN, "CROSS_CFMEDIADWELLTIMESIGNAL_15M": NaN, "CROSS_CFMEDIADWELLTIMESIGNAL_1H": NaN, "CROSS_CFMEDIADWELLTIMESIGNAL_6H": NaN, "CROSS_CFMEDIADWELLTIMESIGNAL_1D": NaN, "CROSS_CFMEDIADWELLTIMESIGNAL_7D": NaN, "CROSS_CFMEDIADWELLTIMESIGNAL_30D": NaN, "CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_1M": NaN, "CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_5M": NaN, "CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_15M": NaN, "CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_1H": NaN, "CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_6H": NaN, "CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_1D": NaN, "CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_7D": NaN, "CROSS_CFMEDIATAPPEDTAKENBYSIGNAL_30D": NaN, "CROSS_CFMEDIAVIEWEDSIGNAL_1M": NaN, "CROSS_CFMEDIAVIEWEDSIGNAL_5M": NaN, "CROSS_CFMEDIAVIEWEDSIGNAL_15M": NaN, "CROSS_CFMEDIAVIEWEDSIGNAL_1H": NaN, "CROSS_CFMEDIAVIEWEDSIGNAL_6H": NaN, "CROSS_CFMEDIAVIEWEDSIGNAL_1D": NaN, "CROSS_CFMEDIAVIEWEDSIGNAL_7D": NaN, "CROSS_CFMEDIAVIEWEDSIGNAL_30D": NaN}]'
features_dict = json.loads(js)
features_dict

[{'TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_1M': nan,
  'TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_5M': nan,
  'TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_15M': nan,
  'TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_1H': nan,
  'TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_6H': nan,
  'TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_1D': nan,
  'TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_7D': nan,
  'TAKENBY_CFMEDIAADDEDCOMMENTSIGNAL_30D': nan,
  'TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_1M': 7.696535994627496e-70,
  'TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_5M': 2.618683775587088e-14,
  'TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_15M': 4.7139070665777155e-05,
  'TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_1H': 0.13935345825584178,
  'TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_6H': 1.3060443652450713,
  'TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_1D': 4.087081774077868,
  'TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_7D': 24.170414981516267,
  'TAKENBY_CFMEDIAADDEDREACTIONSIGNAL_30D': 47.9210113498678,
  'TAKENBY_CFMEDIADWELLTIMESIGNAL_1M': 8.538857909291201e-05,
  'TAKENBY_CFMEDIADWELLTIMESIGNAL_5M': 0.32844

In [35]:
dt = pd.DataFrame(json.loads(js))
dmatrix = xgb.DMatrix(dt)
predictions = bst.predict(dmatrix)
predictions

array([0.02219246, 0.03387076], dtype=float32)

In [38]:
dt = pd.DataFrame(json.loads(js))
dmatrix = xgb.DMatrix(dt)
predictions = bst.predict(dmatrix)
predictions

array([0.01806126, 0.02934477], dtype=float32)